# Function to set up text alert messages

This is a scratch notebook to develop a script for messaging people about a spike at a sensor near them

## Import Packages

In [1]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times

# Database 

import psycopg2
from psycopg2 import sql

# Analysis

import numpy as np
import geopandas as gpd
import pandas as pd

# Get CWD

cwd = os.getcwd()

In [2]:
## Database Credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'Scripts', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].rstrip('\n').split(', ')

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

## New Alert

In [3]:
# Dummy variables

sensor_indices = [145604, 145604]
readings = [32.7, 32.7]
intersection_indices = [203, 315]


In [4]:
# Get direction and distance from intersection they signed up for
# Using PostGIS

conn = psycopg2.connect(**pg_connection_dict)

# Create json cursor
cur = conn.cursor()

# Get the example as a Geojson
cmd = sql.SQL('''
WITH sensor_data as
(
SELECT * 
FROM "PurpleAir Stations" p
JOIN unnest({}::int[])
WITH ORDINALITY t(sensor_index, ord)
USING (sensor_index)
ORDER BY t.ord
),
intersection_data as
(
SELECT *
FROM "Road Intersections" i
JOIN unnest({}::int[])
WITH ORDINALITY t(intersection_index, ord)
USING (intersection_index)
ORDER BY t.ord
)
SELECT degrees(ST_Azimuth(ST_Transform(i.geometry, 26915), ST_Transform(p.geometry,26915))), -- https://gis.stackexchange.com/questions/54427/how-to-find-out-direction-postgis
ST_Distance(ST_Transform(i.geometry, 26915), ST_Transform(p.geometry,26915)),
i."NS_cross_street",
i."EW_cross_street", i.ord
FROM sensor_data p
INNER JOIN intersection_data i ON (p.ord = i.ord);
''').format(sql.Literal(sensor_indices), sql.Literal(intersection_indices))

cur.execute(cmd) # Execute

conn.commit() # Committ command

response = cur.fetchall()

In [5]:
response

[(313.3178047527333, 1164.482361384622, '31st Ave S', 'E 26th St', 1),
 (270.90272804702454, 246.6969441842309, '26th Ave S', 'E Franklin Ave', 2)]

In [7]:
# Will return some values that we will unpack...

degree, distance, ns_cross_street, ew_cross_street, order = response[0]

In [8]:
def degrees_to_cardinal(degree):
    '''
    note: this is highly approximate...
    from https://gist.github.com/RobertSudwarts/acf8df23a16afdb5837f
    '''
    dirs = np.array(["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE",
                     "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"], dtype='U')
    ix = int(np.round(degree / (360. / len(dirs)))) % 16
    return dirs[ix]

In [9]:
direction_string = degrees_to_cardinal(degree)

In [11]:
message = f'''SPIKE ALERT! Sensor {sensor_indices[order]} is reading at {readings[order]} micrograms/meter^3.
This sensor is about {round(distance)} meters {direction_string} from the intersection of {ew_cross_street} and {ns_cross_street}.
You are receiving this text because you signed up for SpikeAlerts. 
Please reply with STOP to be removed from this list.'''

In [12]:
print(message)

SPIKE ALERT! Sensor 145604 is reading at 32.7 micrograms/meter^3.
This sensor is about 1164 meters NW from the intersection of E 26th St and 31st Ave S.
You are receiving this text because you signed up for SpikeAlerts. 
Please reply with STOP to be removed from this list.


## End Alert

In [15]:
# Dummy Variables

sensor_index = 168327
duration = 60
max_reading = 50

In [16]:
message = f'''Spike alert for sensor {sensor_index} is over.
This event lasted {duration} minutes with a maximum value of {max_reading} micrograms/meter^3.
You are receiving this text because you signed up for SpikeAlerts. 
Please reply with STOP to be removed from this list.'''

In [17]:
print(message)

Spike alert for sensor 168327 is over.
This event lasted 60 minutes with a maximum value of 50 micrograms/meter^3.
You are receiving this text because you signed up for SpikeAlerts. 
Please reply with STOP to be removed from this list.
